In [ ]:
import logging
import sys

root = logging.getLogger()
root.setLevel(logging.DEBUG)
handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter(
    "%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
handler.addFilter(logging.Filter("trulens"))
handler.setFormatter(formatter)
root.addHandler(handler)

In [ ]:
# Create snowpark session.

from snowflake.snowpark import Session
from trulens.connectors.snowflake import SnowflakeConnector

snowflake_connection_parameters = {
    "account": "SNOWHOUSE",
    "user": "dhuang",
    "database": "AIML",
    "schema": "INFORMATION_SCHEMA",
    "authenticator": "externalbrowser",
}
snowpark_session = Session.builder.configs(
    snowflake_connection_parameters
).create()

sf_connector = SnowflakeConnector(snowpark_session=snowpark_session)

In [ ]:
snowpark_session.sql("")

In [ ]:
# Simplest Virtual Run approach!
import uuid

from trulens.apps.app import TruApp
from trulens.core.run import RunConfig

APP_NAME = "Evaluation Run on existing data"
APP_VERSION = "V1"

# Create TruApp with None - no app object or main_method needed!
tru_app = TruApp(
    app=None,  # No app object needed for virtual runs
    app_name=APP_NAME,
    app_version=APP_VERSION,
    connector=sf_connector,
)

# Create run config with dataset specification
run_name = f"virtual_run_{uuid.uuid4()}"

run_config = RunConfig(
    run_name=run_name,
    dataset_name="dhuang.virtual_test.virtual_run_test",  # Your Snowflake table name
    source_type="TABLE",
    dataset_spec={
        # The dataset_spec maps span attribute paths to column names
        # This creates spans dynamically based on what you define here!
        "record_root.input": "QUERY_STRING",  # Root span input
        "record_root.output": "OUTPUT_STRING",  # Root span output
        "retrieval.retrieved_contexts": "CONTEXTS",  # Retrieval span contexts
        "record_root.ground_truth_output": "EXPECTED_RESPONSE",  # Ground truth output
    },
)

# Use the existing add_run() flow
virtual_run = tru_app.add_run(run_config=run_config)

print(f"Created virtual run: {run_name}")

In [ ]:
virtual_run.describe()

In [ ]:
# Start the virtual run - this will create OTEL spans from existing data
virtual_run.start(virtual=True)

With the new virtual run feature, you can now ingest existing data directly into the Event Table and compute metrics on them. 
The example schema used is shown below:
```sql
create table YOUR_TABLE_NAME (
    query_string VARCHAR,
    output_string VARCHAR, 
    contexts VARCHAR,
    expected_response VARCHAR
);
```


In [ ]:
# Check virtual run status
import time

while virtual_run.get_status() == "INVOCATION_IN_PROGRESS":
    print("Waiting for ingestion to complete...")
    time.sleep(2)

print(f"Virtual run status: {virtual_run.get_status()}")

## Compute Metrics

Computes the RAG triad metrics for both runs to measure the quality of response in the RAG application.

In [ ]:
# Compute metrics for the virtual run
virtual_run.compute_metrics(["correctness"])

In [ ]:
virtual_run.get_status()

## Evaluation Results

To view evaluation results:
* Login to [Snowsight](https://app.snowflake.com/).
* Navigate to **AI & ML** -> **Evaluations** from the left navigation menu.
* Select “RAG evaluation run on existing data” to view the runs, see detailed traces and compare runs.

#### You can also view the records and evaluation results using SDK `get_records_and_feedback`

In [ ]:
df, metrics_list = tru_app.session.get_records_and_feedback(
    app_name=APP_NAME, app_version=APP_VERSION
)

In [ ]:
df